
# Projet Maison 2

In [2]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [3]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")

# A COMPLETER avec les colonnes
# - lat, lon : latitude et longitude des communes

#geo["Latitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[0])).astype(float)
#geo["Longitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[1])).astype(float)
geo[["lat","lon"]] = geo["geo_point_2d"].str.split(", ", expand=True).astype(float)

# - cp_ville : Code Postal + " " + "Commune"

geo["cp_ville"] = geo["Code Postal"]+ " " + geo["Commune"]
#geo2=geo[["Latitude","Longitude","cp_ville"]]
geo.head()
#geo.info()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,lat,lon,cp_ville
Code INSEE,,,,,,,,,,,,,
31080,31350,BOULOGNE-SUR-GESSE,HAUTE-GARONNE,MIDI-PYRENEES,Chef-lieu canton,301.0,2470.0,1.6,"43.2904403081, 0.650641474176","{""type"": ""Polygon"", ""coordinates"": [[[0.671852...",43.290440,0.650641,31350 BOULOGNE-SUR-GESSE
11143,11510,FEUILLA,AUDE,LANGUEDOC-ROUSSILLON,Commune simple,314.0,2426.0,0.1,"42.9291375888, 2.90138923544","{""type"": ""Polygon"", ""coordinates"": [[[2.906996...",42.929138,2.901389,11510 FEUILLA
43028,43200,BESSAMOREL,HAUTE-LOIRE,AUVERGNE,Commune simple,888.0,743.0,0.4,"45.1306448726, 4.07952494849","{""type"": ""Polygon"", ""coordinates"": [[[4.091674...",45.130645,4.079525,43200 BESSAMOREL
78506,78660,PRUNAY-EN-YVELINES,YVELINES,ILE-DE-FRANCE,Commune simple,155.0,2717.0,0.8,"48.5267627187, 1.80513972814","{""type"": ""Polygon"", ""coordinates"": [[[1.829610...",48.526763,1.805140,78660 PRUNAY-EN-YVELINES
84081,84310,MORIERES-LES-AVIGNON,VAUCLUSE,PROVENCE-ALPES-COTE D'AZUR,Commune simple,49.0,1042.0,7.6,"43.9337788848, 4.90875878315","{""type"": ""Polygon"", ""coordinates"": [[[4.927176...",43.933779,4.908759,84310 MORIERES-LES-AVIGNON


**Partie A**

- Compléter le chargement des données en ajoutant au dataframe `GEO`
    - les colonnes "lat" et "lon" avec la latitude et la longitude des communes
    - une colonne "cp_ville" avec le Code Postal + un espace + et le nom de la Commune
- Ecrire une fonction `search_city(lat, lon)` qui retourne le "cp_ville" de la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ecrire une fonction `dms2dec(deg, min, sec)` qui convertit les degrés, minutes, secondes en valeur numérique pour pouvoir utiliser la fonction précédente avec un GPS.

In [5]:
# fonction recherche de ville

def search_city(lat, long):
    loc1=(lat, long)
    #X=geo[["Latitude"]["Longitude"]]
    geo["Values"] = geo.apply(lambda x:np.sqrt(np.square(x.lat-lat)+np.square(x.lon-long)), axis = 1)
    result=geo["Values"].idxmin()
    return geo["cp_ville"][result]

#loc1=(28.426846,77.088834)
#loc2=(28.394231,77.050308)

In [6]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
  return deg+ mn/60 + sec/3600


In [7]:
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = np.random.uniform(a, b)
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
lon = np.random.uniform(a, b)

search_city(lat, lon)

'71330 BOSJEAN'

In [8]:
# à partir de coordonnées GPS précises
search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'91120 PALAISEAU'

**Partie B**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python **json** pour parser les valeurs de la colonne "geo_shape" et mettre le résultat (`Series`) dans la variable `GEO_SHAPE`.
- Ecrire une fonction `get_types()` qui retourne le décompte (`value_counts()`) des valeurs accédées avec la clé "type".
- Ecrire une fonction `get_coordinates_len()` qui retourne le décompte (`value_counts()`) des longueurs des listes accédées avec la clé "coordinates".
- Ecrire une fonction `get_most_complex_city()` qui retourne la commune est constituée du plus grand nombre de polygones ?
- Ecrire une fonction `get_nb_cities_2_polygons()` qui retourne  le nombre de villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- **Facultatif :**
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : on pourra installer la librairie **shapely**, utiliser la classe Polygon de **shapely.geometry**  et la méthode `contains()`. Sur Windows **shapely** peut nécessiter d'installer manuellement la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [107]:
# On utilise la librairie Python json pour parser les valeurs 
# de la colonne "geo_shape" et mettre le résultat (Series) dans la variable GEO_SHAPE

import json
# La variable GEO_SHAPE doit contenir une Serie
# correspondant aux valeurs de la colonne "geo_shape" parsées avec la librairie json
GEO_SHAPE = pd.Series(dtype=object)
GEO_SHAPE=geo["geo_shape"].apply(json.loads).apply(pd.Series)

GEO_SHAPE

,type,coordinates
Code INSEE,,
31080,Polygon,"[[[0.671852008023359, 43.268343740258835], [0...."
11143,Polygon,"[[[2.906996888482618, 42.895679017325214], [2...."
43028,Polygon,"[[[4.091674013520042, 45.10387299771701], [4.0..."
78506,Polygon,"[[[1.82961019583912, 48.496571987531524], [1.8..."
84081,Polygon,"[[[4.927176726246697, 43.90528366187598], [4.8..."
80462,Polygon,"[[[1.849459801983121, 50.17500092547019], [1.8..."
40200,Polygon,"[[[-0.710168187746476, 44.34474055229595], [-0..."
17394,Polygon,"[[[-0.66124517570293, 46.112107175022146], [-0..."
...,...,...


In [199]:
# Ecrivons une fonction get_types() 
# qui retourne le décompte (value_counts()) des valeurs accédées avec la clé "type".


def get_types():
  return GEO_SHAPE["type"].value_counts()
get_types()

Polygon         36670
MultiPolygon       72
Name: type, dtype: int64

In [200]:
# Ecrivons une fonction get_coordinates_len() qui retourne le décompte (value_counts())
#des longueurs des listes accédées avec la clé "coordinates".

def get_coordinates_len():
     geo["data"]=GEO_SHAPE['coordinates'].apply(lambda x:np.array(x,dtype=object)).apply(lambda y: np.shape(y)[0])
     return geo["data"].value_counts()
GEO_SHAPE["res"]=geo["data"]

get_coordinates_len()

1    36660
2       80
3        1
4        1
Name: data, dtype: int64

In [186]:
# Ecrivons une fonction get_most_complex_city() 
#qui retourne la commune est constituée du plus grand nombre de polygones ?
def get_most_complex_city():
  value=geo["data"].idxmax()
  return geo["cp_ville"][value]

get_most_complex_city()

'83400 HYERES'

In [197]:
# Ecrivons une fonction get_nb_cities_2_polygons() qui retourne le nombre de villes qui 
#sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?

def get_nb_cities_2_polygons():
    res=geo["cp_ville"].loc[(GEO_SHAPE["type"] == "Polygon") & (GEO_SHAPE["res"] ==2)]
    return len(res)

get_nb_cities_2_polygons()

10

In [119]:
# tests
import unittest

class Session2Test(unittest.TestCase):
    
    def test_partie_A1(self):
        # on applique la fonction cherche_ville() à une coordonnée tirée au hasard
        np.random.seed(0)
        a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
        lat = np.random.uniform(a, b)
        a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
        lon = np.random.uniform(a, b)

        cp_ville = search_city(lat, lon)
        self.assertEqual(cp_ville, "71330 BOSJEAN")
        
    def test_partie_A2(self):
        # à partir de coordonnées GPS précises
        cp_ville = search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))
        self.assertEqual(cp_ville, "91120 PALAISEAU")
        
    def test_partie_B1(self):
        # check types
        dico = get_types()
        self.assertEqual(dico["Polygon"], 36670)
        self.assertEqual(dico["MultiPolygon"], 72)
        
    def test_partie_B2(self):
        # check coordinates len
        dico = get_coordinates_len()
        self.assertEqual(dico[1], 36660)
        self.assertEqual(dico[2], 80)
       
    def test_partie_B3(self):
        # check most complex city
        cp_ville = get_most_complex_city()
        self.assertEqual(cp_ville, "83400 HYERES")
        
    def test_partie_B4(self):
        # check nb cities 2 polygons
        nb = get_nb_cities_2_polygons()
        self.assertEqual(nb, 10)

In [201]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session2Test)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)
    
run_tests()

test_partie_A1 (__main__.Session2Test) ... ok
test_partie_A2 (__main__.Session2Test) ... ok
test_partie_B1 (__main__.Session2Test) ... ok
test_partie_B2 (__main__.Session2Test) ... ok
test_partie_B3 (__main__.Session2Test) ... ok
test_partie_B4 (__main__.Session2Test) ... ok

----------------------------------------------------------------------
Ran 6 tests in 2.416s

OK
